In [1]:
# Entendiendo Memoria en LLMs


# Entendiendo Memoria en LLMs

En los cuadernos anteriores, exploramos con éxito cómo los modelos de OpenAI pueden mejorar los resultados de las consultas de Azure AI Search. 

Sin embargo, aún tenemos que descubrir cómo entablar una conversación con el LLM. Con [Bing Chat](http://chat.bing.com/), por ejemplo, esto es posible, ya que puede entender y hacer referencia a las respuestas anteriores.

Existe la idea errónea de que los LLM (Large Language Models) tienen memoria. Esto no es cierto. Aunque poseen conocimientos, no retienen información de preguntas anteriores que se les hayan formulado.

En este Cuaderno, nuestro objetivo es ilustrar cómo podemos "dotar de memoria" a los LLM de forma eficaz empleando indicaciones y contexto.

In [1]:
import os
import random
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from typing import List

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import CustomAzureSearchRetriever#, get_answer
from common.prompts import DOCSEARCH_PROMPT

from dotenv import load_dotenv
load_dotenv("credentials.env")

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
print(os.environ["OPENAI_API_VERSION"])

2023-12-01-preview


### Empecemos por lo básico
Vamos a utilizar un ejemplo muy simple para ver si el modelo GPT de Azure OpenAI tiene memoria. De nuevo usaremos langchain para simplificar nuestro código 

In [3]:
QUESTION = 'dame los detalles del ticket 3763?'
FOLLOW_UP_QUESTION = "haz una lista"

In [4]:
COMPLETION_TOKENS = 1000
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [5]:
# We create a very simple prompt template, just the question as is:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "- **You MUST ONLY answer the question from information contained in the extracted parts (CONTEXT) below**, DO NOT use your prior knowledge. You are an advanced language model specialized in cybersecurity, with deep and up-to-date knowledge on security practices, threats, and technologies. Your goal is to provide precise and useful responses based on the information contained in the ingested documents. This information includes sales and opportunity data from Salesforce, operational details from ServiceDesk+, and data on assets managed by the cybersecurity company"),
    ("user", "{input}")
])

In [6]:
# Let's see what the GPT model responds
chain = prompt | llm | output_parser
response_to_initial_question = chain.invoke({"input": QUESTION})
display(Markdown(response_to_initial_question))

Lo siento, pero no tengo acceso directo a los detalles específicos del ticket 3763. Sin embargo, si proporcionas información relevante contenida en los documentos que he procesado, puedo ayudarte a analizarla y proporcionarte información basada en esos datos.

In [7]:
#Now let's ask a follow up question
printmd(chain.invoke({"input": FOLLOW_UP_QUESTION}))

Lamentablemente, no puedo cumplir con tu solicitud ya que no proporcionaste suficiente información sobre el tipo de lista que necesitas. Por favor, proporciona detalles adicionales o haz una pregunta más específica para que pueda ayudarte de manera efectiva.

Como puedes ver, no recuerda lo que acaba de responder, a veces responde basado solo en el prompt del sistema, o simplemente al azar. Esto prueba que la LLM NO tiene memoria y que necesitamos dar la memoria como un historial de conversación como parte del prompt, así:

In [8]:
hist_prompt = ChatPromptTemplate.from_template(
"""
    {history}
    Human: {question}
    AI:
"""
)
chain = hist_prompt | llm | output_parser

In [9]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response_to_initial_question)

In [10]:
printmd(chain.invoke({"history":Conversation_history, "question": FOLLOW_UP_QUESTION}))

Lo siento, pero como modelo de lenguaje de inteligencia artificial, no tengo la capacidad de acceder a información específica sobre tickets o documentos. Mi función es generar respuestas basadas en el conocimiento general que he adquirido. Si necesitas información detallada sobre un ticket específico, te recomendaría consultar directamente la fuente de la que proviene el ticket.

**Bingo**, así que ya sabemos cómo crear un chatbot utilizando LLMs, sólo tenemos que mantener el estado/historial de la conversación y pasarlo como contexto cada vez.

## Ahora que entendemos el concepto de memoria añadiendo la historia como contexto, volvamos a nuestro buscador inteligente GPT

Del sitio web de Langchain:
    
Un sistema de memoria necesita soportar dos acciones básicas: lectura y escritura. Recordemos que cada cadena define un núcleo lógico de ejecución que espera ciertas entradas. Algunas de estas entradas vienen directamente del usuario, pero otras pueden venir de la memoria. Una cadena interactuará con su sistema de memoria dos veces en una ejecución dada.

    DESPUÉS de recibir las entradas iniciales del usuario pero ANTES de ejecutar la lógica del núcleo, una cadena LEERÁ de su sistema de memoria y aumentará las entradas del usuario.
    DESPUÉS de ejecutar la lógica central, pero ANTES de devolver la respuesta, una cadena ESCRIBIRÁ las entradas y salidas de la ejecución actual en la memoria, para que se pueda hacer referencia a ellas en futuras ejecuciones.
    
Así que este proceso añade retrasos a la respuesta, pero es un retraso necesario :)

![image](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [11]:
index1_name = "cogsrch-index-kiografia-csv"
indexes = [index1_name]

In [12]:
# en la terminal
# dar chmod +x al archivo 
# sudo ./download_odbc_driver.sh

In [13]:
# Initialize our custom retriever 
retriever = CustomAzureSearchRetriever(indexes=indexes, topK=5, reranker_threshold=1)

Si te fijas bien en prompts.py, hay una variable opcional en el `DOCSEARCH_PROMPT` llamada `history`. Ahora es el momento de usarla. Es básicamente un marcador de posición donde inyectaremos la conversación en el prompt para que el LLM sea consciente de ello antes de responder.

In [14]:
store = {} # Our first memory will be a dictionary in memory

# We have to define a custom function that takes a session_id and looks somewhere
# (in this case in a dictionary in memory) for the conversation
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [16]:
# We use our original chain with the retriever but removing the StrOutputParser
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "history": itemgetter("history")
    }
    | DOCSEARCH_PROMPT
    | llm
)

## Then we pass the above chain to another chain that adds memory to it

output_parser = StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
) | output_parser

In [19]:
# This is where we configure the session id
config={"configurable": {"session_id": "abc123_KIO_1"}}

Fíjate que estamos añadiendo una variable `history` en la llamada. Esta variable contendrá la historia del chat dentro del prompt.

In [20]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Parent run bc1118ec-e54e-4748-bc26-fcb7dbf60395 not found for run 9f00278e-4f54-44aa-9863-2af9090d273b. Treating as a root run.


Lo siento, no tengo información sobre el ticket con el ID 3763 en los datos proporcionados.

In [27]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Parent run 7c15987a-08a7-4e0b-ab35-b0b041172c13 not found for run 50ce17a9-f10e-458e-a218-684fed1268e8. Treating as a root run.


Tengo información de 2 clientes en total. Los nombres de estos clientes son:
1. Bulkmatic
2. Procesar

In [24]:
# Remembers
printmd(chain_with_history.invoke({"question": "Dame la lista completa de clientes"},config=config))

Parent run 60841736-a506-493f-b597-3fd86892be26 not found for run 3dc18f44-8d77-4373-bf11-cf2b6d8218db. Treating as a root run.


Tengo información de 2 clientes en total. Los nombres de estos clientes son:
1. Bulkmatic
2. Procesar

## Usando CosmosDB como memoria persistente

En la celda anterior hemos añadido memoria RAM local a nuestro chatbot. Sin embargo, no es persistente, se elimina una vez que la sesión del usuario de la aplicación se termina. Es necesario entonces utilizar una Base de Datos para el almacenamiento persistente de cada una de las conversaciones de los usuarios del bot, no sólo para Análisis y Auditoría, sino también si deseamos proporcionar recomendaciones en el futuro. 

Aquí almacenaremos el historial de conversaciones en CosmosDB para futuros propósitos de auditoría.
Utilizaremos una clase en LangChain llamada CosmosDBChatMessageHistory

In [21]:
# Create the function to retrieve the conversation

def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [22]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
) | output_parser

In [23]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}

In [24]:
config

{'configurable': {'session_id': 'session248', 'user_id': 'user643'}}

In [25]:
printmd(chain_with_history.invoke({"question": QUESTION}, config=config))

Parent run db625af8-a9ec-4a54-a4ef-8e2c4561e4a0 not found for run ab421f59-af78-43c6-9efe-ab406b9b545d. Treating as a root run.


Poseo información de 4 clientes en total. Los nombres de estos clientes son:
1. Abilia
2. Bulkmatic
3. Grupo Zapata
4. Procesar
5. Toka

In [26]:
# Remembers
printmd(chain_with_history.invoke({"question": FOLLOW_UP_QUESTION},config=config))

Parent run a701b6e4-5d08-4bcd-be19-2d74ae63b143 not found for run 32dcc33b-b844-44dc-85ae-be51fdbf18d0. Treating as a root run.


Poseo información de 5 clientes en total. Los nombres de estos clientes son:

1. Abilia
2. Bulkmatic
3. Grupo Zapata
4. Procesar
5. Toka

In [27]:
# Remembers
printmd(chain_with_history.invoke(
    {"question": "cuántos tickets tiene el cliente Bulkmatic?"},
    config=config))

Parent run afc27b90-382e-447d-9218-2b964d7051ee not found for run 3b0ff0d7-2018-4a93-9817-e2c3bdcf5be5. Treating as a root run.


El cliente Bulkmatic tiene un total de 1 ticket en el registro.

In [28]:
try:
    printmd(chain_with_history.invoke(
    {"question": "cuál es el top 5 de tickets de Toka que han tenido el mayor tiempo abierto desde su fecha de creación hasta su fecha de finalización?"},
    config=config))
except Exception as e:
    print(e)

Parent run 65106c54-87dc-42c7-91d7-f0e1b8a9b0f1 not found for run 43c46276-7117-47fe-b0d7-d0184c09d1c3. Treating as a root run.


No se proporciona información suficiente en los documentos para determinar el top 5 de tickets de Toka que han tenido el mayor tiempo abierto desde su fecha de creación hasta su fecha de finalización.

In [29]:
printmd(chain_with_history.invoke(
    {"question": "qué tecnologías tenemos para el cliente Grupo Zapata?"},
    config=config))

Parent run 5340c34c-f499-429a-a32d-cdd6227f669f not found for run f01812a4-6fa1-4266-8835-ede83f4b2aca. Treating as a root run.


Para el cliente Grupo Zapata, disponemos de las siguientes tecnologías:

1. Contivity Vpn Client
2. Firewall

In [30]:
printmd(chain_with_history.invoke(
    {"question": "y qué marcas y modelos se tienen para esas tecnologías?"},
    config=config))

Parent run f13e386a-0589-4ba4-b903-7b90d8fbe595 not found for run 8ae53ca5-8dae-4479-bcbe-4a135ed35e44. Treating as a root run.


No se proporciona información específica sobre las marcas y modelos de las tecnologías "Contivity Vpn Client" y "Firewall" para el cliente Grupo Zapata en los documentos disponibles.

# Resumen
##### Añadir memoria a nuestra aplicación permite al usuario mantener una conversación, sin embargo esta característica no es algo que venga con el LLM, sino que la memoria es algo que debemos proporcionar al LLM en forma de contexto de la pregunta.

Añadimos memoria persistente usando CosmosDB.
También podemos notar que la cadena actual que estamos usando es inteligente, pero no tanto. Aunque le hemos dado memoria, busca documentos similares cada vez, independientemente de la entrada. Esto no parece eficiente, pero a pesar de todo, estamos muy cerca de terminar nuestro primer RAG-talk to your data Bot.

## <u>Nota Importante</u>:<br>

**GPT-3.5-Turbo** puede compararse con un niño de 7 años. Se le pueden dar instrucciones concisas, pero a veces le cuesta seguirlas con precisión (no es demasiado fiable). Además, su "memoria" limitada (contexto simbólico) puede dificultar las conversaciones sostenidas. Sus respuestas también son simples, no profundas.

El **GPT-4-Turbo** muestra las capacidades de un niño de 10-12 años. Posee una mayor capacidad de razonamiento, sigue sistemáticamente las instrucciones y sus respuestas son mejores. Ha ampliado la retención de memoria (mayor tamaño del contexto) para las instrucciones, y destaca en el seguimiento de las mismas. Sus respuestas son profundas y minuciosas.
